In [ ]:
# SEQUENCE MODELS

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

!rm -r aclImdb/train/unsup

import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir/category/fname, val_dir/category/fname)


batch_size=32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)

text_only_train_ds = train_ds.map(lambda x, y: x)



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9223k      0  0:00:08  0:00:08 --:--:-- 16.8M
Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
#Bidirectional RNN
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_length)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 600)         0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               162048    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 162,113
Trainable params: 162,113
Non-trainable params: 0
_______________________________________________________

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras") 
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 42s 53ms/step - loss: 0.5839 - accuracy: 0.6985 - val_loss: 0.5097 - val_accuracy: 0.7908
Epoch 2/10
625/625 [==============================] - 32s 51ms/step - loss: 0.4890 - accuracy: 0.7970 - val_loss: 0.4222 - val_accuracy: 0.8210
Epoch 3/10
625/625 [==============================] - 33s 52ms/step - loss: 0.4470 - accuracy: 0.8122 - val_loss: 0.4264 - val_accuracy: 0.8342
Epoch 4/10
625/625 [==============================] - 33s 53ms/step - loss: 0.4247 - accuracy: 0.8234 - val_loss: 0.3960 - val_accuracy: 0.8308
Epoch 5/10
625/625 [==============================] - 32s 52ms/step - loss: 0.4071 - accuracy: 0.8293 - val_loss: 0.3947 - val_accuracy: 0.8304
Epoch 6/10
625/625 [==============================] - 32s 51ms/step - loss: 0.4000 - accuracy: 0.8359 - val_loss: 0.4234 - val_accuracy: 0.8210
Epoch 7/10
625/625 [==============================] - 32s 51ms/step - loss: 0.3891 - accuracy: 0.8418 - val_loss: 0.3845 - val_accuracy:

2 ways to use word embeddings: Adding it as a layer and learning embeddings with models, or using pretrained word embeddings

In [ ]:
#adding as a layer

inputs = keras.Input(shape=(None, ), dtype="int64")
embeddings = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embeddings)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm.keras") 
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 35s 51ms/step - loss: 0.4601 - accuracy: 0.7946 - val_loss: 0.3372 - val_accuracy: 0.8692
Epoch 2/10
625/625 [==============================] - 33s 53ms/step - loss: 0.3046 - accuracy: 0.8903 - val_loss: 0.3261 - val_accuracy: 0.8798
Epoch 3/10
625/625 [==============================] - 31s 50ms/step - loss: 0.2415 - accuracy: 0.9157 - val_loss: 0.4764 - val_accuracy: 0.8302
Epoch 4/10
625/625 [==============================] - 32s 52ms/step - loss: 0.2002 - accuracy: 0.9313 - val_loss: 0.3069 - val_accuracy: 0.8810
Epoch 5/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1697 - accuracy: 0.9441 - val_loss: 0.3299 - val_accuracy: 0.8856
Epoch 6/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1434 - accuracy: 0.9527 - val_loss: 0.3837 - val_accuracy: 0.8672
Epoch 7/10
625/625 [==============================] - 32s 51ms/step - loss: 0.1162 - accuracy: 0.9633 - val_loss: 0.3751 - val_accuracy:

In [ ]:
#using masking

inputs = keras.Input(shape=(None, ), dtype="int64")
embeddings = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embeddings)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_mask.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm_mask.keras") 
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 50s 68ms/step - loss: 0.3995 - accuracy: 0.8201 - val_loss: 0.3504 - val_accuracy: 0.8510
Epoch 2/10
625/625 [==============================] - 37s 59ms/step - loss: 0.2297 - accuracy: 0.9119 - val_loss: 0.2942 - val_accuracy: 0.8798
Epoch 3/10
625/625 [==============================] - 40s 64ms/step - loss: 0.1648 - accuracy: 0.9387 - val_loss: 0.3182 - val_accuracy: 0.8872
Epoch 4/10
625/625 [==============================] - 36s 58ms/step - loss: 0.1264 - accuracy: 0.9552 - val_loss: 0.3334 - val_accuracy: 0.8784
Epoch 5/10
625/625 [==============================] - 37s 59ms/step - loss: 0.0890 - accuracy: 0.9690 - val_loss: 0.3843 - val_accuracy: 0.8796
Epoch 6/10
625/625 [==============================] - 38s 61ms/step - loss: 0.0694 - accuracy: 0.9758 - val_loss: 0.4016 - val_accuracy: 0.8820
Epoch 7/10
625/625 [==============================] - 36s 58ms/step - loss: 0.0476 - accuracy: 0.9833 - val_loss: 0.4702 - val_accuracy:

In [ ]:
#Using Pretrained 
#2 popular options: Word2Vec and GloVe

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-07-19 17:02:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-07-19 17:02:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-07-19 17:02:55--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
#Parsing the glove embedding file

import numpy as np
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {} 
with open(path_to_glove_file) as f:
 for line in f:
   word, coefs = line.split(maxsplit=1)
   coefs = np.fromstring(coefs, "f", sep=" ")
   embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [ ]:
#Create GloVe embedding matrix

embedding_dim = 100
vocabulary = text_vectorization.get_vocabulary() 
word_index = dict(zip(vocabulary, range(len(vocabulary)))) 
embedding_matrix = np.zeros((max_tokens, embedding_dim)) 
for word, i in word_index.items():
 if i < max_tokens:
   embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None: 
   embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(max_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                   trainable=False,
                                   mask_zero=True)


In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________